In [1]:
import re
import tensorflow as tf
import numpy as np
import time
import os
import collections
import nltk
import random
import time
import pickle

In [2]:
def clearstring(string):
    string = re.sub('[^A-Za-z\- ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = [y.strip() for y in string]
    return [y.lower() for y in string if len(y) > 3 and y.find('nbsp') < 0]

def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

data_index = 0

# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    global data
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            for word in data[:span]:
                buffer.append(word)
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [3]:
with open('books/chamber.txt', 'r',encoding = "ISO-8859-1") as fopen:
    vocabulary = clearstring(' '.join(nltk.word_tokenize(fopen.read())))
print('example 10 words:',vocabulary[:10])
print('size corpus:',len(vocabulary))
vocabulary_size = len(list(set(vocabulary)))
print('size of unique words:',vocabulary_size)
dimension = 128
skip_window = 1
num_skips = 2
batch_size = 64
location = os.getcwd()

example 10 words: ['first', 'time', 'argument', 'broken', 'over', 'breakfast', 'number', 'four', 'privet', 'drive']
size corpus: 49804
size of unique words: 6831


In [4]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('harry', 1637), ('said', 1207), ('that', 721), ('they', 526)]
Sample data [102, 41, 2634, 717, 21, 610, 1233, 386, 1200, 1149] ['first', 'time', 'argument', 'broken', 'over', 'breakfast', 'number', 'four', 'privet', 'drive']


In [5]:
valid_size = 16
valid_window = 100
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [6]:
graph = tf.Graph()

with graph.as_default():

    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/cpu:0'):
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, dimension], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, dimension],
                                                      stddev=1.0 / np.sqrt(dimension)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                             biases=nce_biases,
                                             labels=train_labels,
                                             inputs=embed,
                                             num_sampled=batch_size / 2,
                                             num_classes=vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        # Compute the cosine similarity between minibatch examples and all embeddings.
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()

num_steps = 200000

In [7]:
with tf.Session(graph=graph) as session:
    init.run()
    print('Initialized')

    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  111.759536743
Nearest to still: bedclothes, occasions, crabbe-, wandlight, uncoiling, enjoyed, bodiless, wwhats,
Nearest to into: knobbly, darkest, disappear, followed, place, records, blocked, jangling,
Nearest to back: anticipating, talismans, bade, force, hairpin, bell, from, wizened,
Nearest to dobby: snigger, slugs, likely, yawning, wonderin, result, macmillan, mumbled,
Nearest to dumbledore: freds, hermiones, reboned, spinnet, solemnly, limb, all-new, dose,
Nearest to hand: burrow, hurtled, laugh, gulps, glittering, rings, scuttled, helping,
Nearest to down: path, house-elves, disarming, famous, stands, dozen, mirrored, bishops,
Nearest to before: spelling, staying, stomach, asked, solitary, sunday, dipped, rifled,
Nearest to very: herbology, footprints, shrunken, alarmed, splash, forms, tumbledown, appointment,
Nearest to long: supplies, memories, tale, been, mightve, wished, cruel, pure,
Nearest to their: cakes, gained, touslehaired, angry

Average loss at step  42000 :  2.43347654909
Average loss at step  44000 :  2.37938748902
Average loss at step  46000 :  2.34639223218
Average loss at step  48000 :  2.3537501201
Average loss at step  50000 :  2.32610395062
Nearest to still: crabbe-, bedclothes, accident, pounce, graciously, arrangements, lazily, exactly,
Nearest to into: greeting, jade-green, blocked, grade, righ, dashed, wrecked, fitted,
Nearest to back: naturally, ourselves, talismans, goblin, resentfully, squealing, forget-me-not, sniggering,
Nearest to dobby: outstretched, considered, forgetting, riddle, malfoys, knees, wasting, tore,
Nearest to dumbledore: reboned, bloody, spinnet, particular, voldemort, half-listening, farther, disappoint,
Nearest to hand: thumbs, measuring, gulps, farewell, hurtled, sleek, further, laugh,
Nearest to down: confidence, mirrored, scampering, replaced, dozen, forcefully, prowled, tartan,
Nearest to before: eral, focused, dipped, panicstricken, checked, admitted, aspect, unavoidable

Average loss at step  92000 :  2.13148775461
Average loss at step  94000 :  2.08285123655
Average loss at step  96000 :  2.11612217891
Average loss at step  98000 :  2.12666891757
Average loss at step  100000 :  2.08642282173
Nearest to still: crabbe-, arrangements, appearance, wwhats, accident, lazily, exactly, hullo,
Nearest to into: righ, greeting, jade-green, downright, grade, wrecked, requested, blocked,
Nearest to back: naturally, ourselves, goblin, talismans, resentfully, sniggering, squealing, press,
Nearest to dobby: forgetting, considered, plumbers, brewing, studies, expelliarmus, outstretched, wasting,
Nearest to dumbledore: reboned, depths, voldemort, wattlebird, towers, harry, scruff, unlocking,
Nearest to hand: thumbs, measuring, promised, gulps, restriction, shouts, comforting, sleek,
Nearest to down: replaced, mirrored, confidence, prowled, tartan, fluttering, explosion, snow-covered,
Nearest to before: eral, barely, focused, aspect, ruddy, dipped, lettering, unavoidabl

Average loss at step  142000 :  2.04947652712
Average loss at step  144000 :  2.03974021906
Average loss at step  146000 :  2.06198832318
Average loss at step  148000 :  2.06294967762
Average loss at step  150000 :  2.02949923196
Nearest to still: crabbe-, appearance, wwhats, arrangements, bewildered, pounce, lazily, known,
Nearest to into: jade-green, greeting, requested, downright, righ, surveying, evaporate, landmark,
Nearest to back: naturally, ourselves, sniggering, goblin, earning, resentfully, thundering, right,
Nearest to dobby: plumbers, rumpled, malice, wasting, considered, studies, riddle, raked,
Nearest to dumbledore: reboned, depths, voldemort, wattlebird, spidersflee, norris, bloody, unlocking,
Nearest to hand: thumbs, promised, measuring, shouts, gulps, restriction, leathery, relieved,
Nearest to down: replaced, mirrored, fluttering, snow-covered, confidence, explosion, prowled, tartan,
Nearest to before: eral, aspect, focused, barely, supposed, dipped, ruddy, supply,
Ne

Average loss at step  192000 :  2.01121841963
Average loss at step  194000 :  2.02866211993
Average loss at step  196000 :  2.04785854423
Average loss at step  198000 :  2.02521161106


In [8]:
with open('chamber-list.p', 'wb') as fopen:
    pickle.dump(list(reverse_dictionary.values()), fopen)

In [9]:
with open('chamber-vector.p', 'wb') as fopen:
    pickle.dump(final_embeddings, fopen)